In [1]:
import spacy
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_dep, visualize_ent
from medspacy.context import ConTextRule
from medspacy.context import ConTextComponent, ConTextItem

import warnings
warnings.filterwarnings('ignore')

In [2]:
nlp = medspacy.load()

In [3]:
nlp.pipe_names

['medspacy_pyrush', 'medspacy_target_matcher', 'medspacy_context']

In [4]:
context = nlp.get_pipe("medspacy_context")
context.categories

{'FAMILY',
 'HISTORICAL',
 'HYPOTHETICAL',
 'NEGATED_EXISTENCE',
 'POSSIBLE_EXISTENCE'}

In [5]:
text = "People with type 2 diabetes often have absense of symptoms at first."
doc = nlp(text)

In [6]:
target_matcher = nlp.get_pipe("medspacy_target_matcher")

In [7]:
target_rules = [
    TargetRule("type X diabetes", "CONDITION",
              pattern=[
                        {"LOWER": 'type'},
                        {"LIKE_NUM": True},
                        {"LOWER": 'diabetes'}]),
    TargetRule("symptoms", "CONDITION"),
]

In [8]:
target_matcher.add(target_rules)

In [9]:
doc = nlp(text)

In [10]:
visualize_ent(doc)

In [11]:
context = ConTextComponent(nlp, rules=None)
context_rules = [ConTextRule(literal="absense of", 
                         category="NEGATED_EXISTENCE", 
                         direction="FORWARD"
                            )]
context.add(context_rules)
context(doc)

People with type 2 diabetes often have absense of symptoms at first.

In [12]:
visualize_ent(doc)

In [13]:
visualize_dep(doc)

In [14]:
for ent in doc.ents:
    if ent._.is_negated:
        print(ent, ent._.is_negated)

symptoms True
